In [1]:
import json
import os
import numpy as np
import matplotlib.pyplot as plt
from moviepy import VideoFileClip
from moviepy.video.fx import Margin
from moviepy.video.fx import AccelDecel
from collections import Counter
from math import floor, ceil

In [2]:
source_directory = 'label-studio-result/'
destination_directory = 'pre-processing-result/'
labels_filename = 'labels.json'
boxes_filename = 'valid-boxes.json'

In [3]:
with open(source_directory+labels_filename, 'r') as file:
    labels = json.load(file)

## 1. Problemas:
    a) Amostras que não estão no escopo de vídeos de CCTV (do inglês, Closed-Circuit Television – Circuito Fechado de Televisão);
    b) Vinhetas.
## 1. Solução: 
    Durante a seleção, esses vídeos foram marcados com o rótulo "lixo" e serão descartados, pois estão fora do escopo definido para o trabalho.

## 2. Problemas:
    a) Trechos com censura;
    b) Trechos com efeitos de transição;
    c) Trechos com indicadores visuais (setas, círculos, etc.).
## 2. Solução:
    Durante a seleção, esses vídeos foram marcados com o rótulo "lixo", por se tratarem de trechos irrecuperáveis ou cuja recuperação causaria mais danos à imagem.

In [4]:
clean_labels = {chave: valor for chave, valor in labels.items() if valor != 'lixo'}

## 3. Problemas
    a) Marcas d’água;
    b) Textos inseridos na imagem;
    c) Censuras.
## 3. Solução
    Recortar a imagem, priorizando o centro do acidente, e descartar as bordas que apresentam esses problemas.

In [5]:
with open(boxes_filename, 'r') as file:
    valid_boxes = json.load(file)

### Houveram algumas revisões nas anotações, por esse motivo alguns arquivos mudaram de nome. Para que todos as anotações estejam com um mesmo nome de arquivo, é necessário um mapeamento.

In [6]:
def load_video(video_params, verify):
    filename = "-".join(video_params['file_upload'].split('-')[1:])
    filename = map_filename(filename)
    if filename in verify:
        return VideoFileClip(source_directory+filename), filename
    else:
        return None, filename

def map_filename(filename):
    try:
        return {
            'ML52jXyFltw_19.mp4': 'ML52jXyFltw_18.mp4',
            'ML52jXyFltw_20.mp4': 'ML52jXyFltw_19.mp4',
            'ML52jXyFltw_21.mp4': 'ML52jXyFltw_20.mp4',
            'ML52jXyFltw_22.mp4': 'ML52jXyFltw_21.mp4',
            'ML52jXyFltw_23.mp4': 'ML52jXyFltw_22.mp4',
            'ML52jXyFltw_24.mp4': 'ML52jXyFltw_23.mp4',
            'Z4dWfjsrYhQ_3.mp4': 'Z4dWfjsrYhQ_2.mp4',
            'Z4dWfjsrYhQ_4.mp4': 'Z4dWfjsrYhQ_3.mp4',
            'Z4dWfjsrYhQ_5.mp4': 'Z4dWfjsrYhQ_4.mp4',
            'Z4dWfjsrYhQ_6.mp4': 'Z4dWfjsrYhQ_5.mp4',
            'Z4dWfjsrYhQ_7.mp4': 'Z4dWfjsrYhQ_6.mp4',
            'Z4dWfjsrYhQ_8.mp4': 'Z4dWfjsrYhQ_7.mp4',
            'Z4dWfjsrYhQ_9.mp4': 'Z4dWfjsrYhQ_8.mp4',
            'Z4dWfjsrYhQ_10.mp4': 'Z4dWfjsrYhQ_9.mp4',
            'Z4dWfjsrYhQ_11.mp4': 'Z4dWfjsrYhQ_10.mp4',
            'Z4dWfjsrYhQ_12.mp4': 'Z4dWfjsrYhQ_11.mp4',
            'Z4dWfjsrYhQ_13.mp4': 'Z4dWfjsrYhQ_12.mp4',
            'Z4dWfjsrYhQ_14.mp4': 'Z4dWfjsrYhQ_13.mp4',
            'Z4dWfjsrYhQ_15.mp4': 'Z4dWfjsrYhQ_14.mp4',
            'Z4dWfjsrYhQ_16.mp4': 'Z4dWfjsrYhQ_15.mp4',
            'Z4dWfjsrYhQ_17.mp4': 'Z4dWfjsrYhQ_16.mp4',
            'Z4dWfjsrYhQ_18.mp4': 'Z4dWfjsrYhQ_17.mp4',
            'Z4dWfjsrYhQ_19.mp4': 'Z4dWfjsrYhQ_18.mp4',
            'Z4dWfjsrYhQ_20.mp4': 'Z4dWfjsrYhQ_19.mp4',
            'Z4dWfjsrYhQ_21.mp4': 'Z4dWfjsrYhQ_20.mp4',
            'Z4dWfjsrYhQ_22.mp4': 'Z4dWfjsrYhQ_21.mp4',
            'Z4dWfjsrYhQ_23.mp4': 'Z4dWfjsrYhQ_22.mp4',
            'Z4dWfjsrYhQ_24.mp4': 'Z4dWfjsrYhQ_23.mp4',
            'Z4dWfjsrYhQ_25.mp4': 'Z4dWfjsrYhQ_24.mp4',
            'Z4dWfjsrYhQ_26.mp4': 'Z4dWfjsrYhQ_25.mp4',
            'Z4dWfjsrYhQ_27.mp4': 'Z4dWfjsrYhQ_26.mp4',
            'Z4dWfjsrYhQ_28.mp4': 'Z4dWfjsrYhQ_27.mp4',
            'Z4dWfjsrYhQ_29.mp4': 'Z4dWfjsrYhQ_28.mp4',
            'Z4dWfjsrYhQ_30.mp4': 'Z4dWfjsrYhQ_29.mp4',
            'Z4dWfjsrYhQ_31.mp4': 'Z4dWfjsrYhQ_30.mp4',
            'V17rwsM9ELw_35.mp4': 'V17rwsM9ELw_35.mp4',
            'V17rwsM9ELw_36.mp4': 'V17rwsM9ELw_35.mp4',
            'V17rwsM9ELw_37.mp4': 'V17rwsM9ELw_36.mp4',
            'V17rwsM9ELw_38.mp4': 'V17rwsM9ELw_37.mp4',
            'V17rwsM9ELw_39.mp4': 'V17rwsM9ELw_38.mp4',
            'V17rwsM9ELw_40.mp4': 'V17rwsM9ELw_39.mp4',
            'V17rwsM9ELw_41.mp4': 'V17rwsM9ELw_40.mp4',
            'V17rwsM9ELw_42.mp4': 'V17rwsM9ELw_41.mp4',
            'V17rwsM9ELw_43.mp4': 'V17rwsM9ELw_42.mp4',
            'V17rwsM9ELw_44.mp4': 'V17rwsM9ELw_43.mp4',
            'V17rwsM9ELw_45.mp4': 'V17rwsM9ELw_44.mp4',
            'V17rwsM9ELw_46.mp4': 'V17rwsM9ELw_45.mp4',
            'V17rwsM9ELw_47.mp4': 'V17rwsM9ELw_46.mp4',
            'V17rwsM9ELw_48.mp4': 'V17rwsM9ELw_47.mp4',
            'V17rwsM9ELw_49.mp4': 'V17rwsM9ELw_48.mp4',
            'V17rwsM9ELw_50.mp4': 'V17rwsM9ELw_49.mp4',
            'V17rwsM9ELw_51.mp4': 'V17rwsM9ELw_50.mp4',
            'upawiSphDM8_6.mp4': 'upawiSphDM8_5.mp4',
            'upawiSphDM8_7.mp4': 'upawiSphDM8_6.mp4',
            'upawiSphDM8_8.mp4': 'upawiSphDM8_7.mp4',
            'upawiSphDM8_9.mp4': 'upawiSphDM8_8.mp4',
            'upawiSphDM8_10.mp4': 'upawiSphDM8_9.mp4',
            'upawiSphDM8_11.mp4': 'upawiSphDM8_10.mp4',
            'upawiSphDM8_12.mp4': 'upawiSphDM8_11.mp4',
            'upawiSphDM8_13.mp4': 'upawiSphDM8_12.mp4',
            'upawiSphDM8_14.mp4': 'upawiSphDM8_13.mp4',
            'upawiSphDM8_15.mp4': 'upawiSphDM8_14.mp4',
            'upawiSphDM8_16.mp4': 'upawiSphDM8_15.mp4',
            'upawiSphDM8_17.mp4': 'upawiSphDM8_16.mp4',
            'upawiSphDM8_18.mp4': 'upawiSphDM8_17.mp4',
            'upawiSphDM8_19.mp4': 'upawiSphDM8_18.mp4',
            'upawiSphDM8_20.mp4': 'upawiSphDM8_19.mp4',
            'upawiSphDM8_21.mp4': 'upawiSphDM8_20.mp4',
            'upawiSphDM8_22.mp4': 'upawiSphDM8_21.mp4',
            'upawiSphDM8_23.mp4': 'upawiSphDM8_22.mp4',
            'upawiSphDM8_24.mp4': 'upawiSphDM8_23.mp4',
            'upawiSphDM8_25.mp4': 'upawiSphDM8_24.mp4',
            'upawiSphDM8_26.mp4': 'upawiSphDM8_25.mp4',
            'upawiSphDM8_27.mp4': 'upawiSphDM8_26.mp4',
            'upawiSphDM8_28.mp4': 'upawiSphDM8_26.mp4',
            'upawiSphDM8_29.mp4': 'upawiSphDM8_27.mp4',
            'upawiSphDM8_30.mp4': 'upawiSphDM8_28.mp4',
            'upawiSphDM8_31.mp4': 'upawiSphDM8_29.mp4',
            'upawiSphDM8_32.mp4': 'upawiSphDM8_30.mp4',
            'upawiSphDM8_33.mp4': 'upawiSphDM8_31.mp4',
            'upawiSphDM8_34.mp4': 'upawiSphDM8_32.mp4',
            'upawiSphDM8_35.mp4': 'upawiSphDM8_33.mp4',
            'upawiSphDM8_36.mp4': 'upawiSphDM8_34.mp4',
            'upawiSphDM8_37.mp4': 'upawiSphDM8_35.mp4',
            'upawiSphDM8_38.mp4': 'upawiSphDM8_36.mp4',
            'upawiSphDM8_39.mp4': 'upawiSphDM8_37.mp4',
            'upawiSphDM8_40.mp4': 'upawiSphDM8_38.mp4',
            'upawiSphDM8_41.mp4': 'upawiSphDM8_39.mp4',
            'upawiSphDM8_42.mp4': 'upawiSphDM8_40.mp4',
            'upawiSphDM8_43.mp4': 'upawiSphDM8_41.mp4',
            'upawiSphDM8_44.mp4': 'upawiSphDM8_42.mp4',
            'upawiSphDM8_45.mp4': 'upawiSphDM8_43.mp4',
            'upawiSphDM8_46.mp4': 'upawiSphDM8_44.mp4'
        }[filename]
    except KeyError:
        return filename

In [7]:
def crop(clip, video_params, filename):
    annotations = video_params['annotations'][0]['result']
    return skip_or_crop(clip, annotations, filename)

def skip_or_crop(clip, annotations, filename):
    if annotations:
        return list_cropped_clis(clip, annotations, filename)
    else:
        return [(clip, filename)]

def list_cropped_clis(clip, annotations, filename):
    cropped_clips = []
    for i, crop_annotation in enumerate(annotations):
        cropped_filename = filename.split(".")[0] + f"_{i}.mp4"
        cropped_clips.append(get_cropped_clip(clip, crop_annotation['value']['sequence'][0], cropped_filename))
    return cropped_clips
    
def get_cropped_clip(clip, crop_area, filename):
    x1, y1, x2, y2 = (
        ceil(clip.size[0]*crop_area['x']/100), 
        ceil(clip.size[1]*crop_area['y']/100), 
        floor(clip.size[0]*(crop_area['x']+crop_area['width'])/100), 
        floor(clip.size[1]*(crop_area['y']+crop_area['height'])/100)
    )
    # x1 = x1 if (x1+x2)%2==0 else x1+1
    # y1 = y1 if (y1+y2)%2==0 else y1+1
    return clip.cropped(x1=x1, y1=y1, x2=x2, y2=y2), filename

In [8]:
def resize(clip, max_width, max_height):
    scale_w = max_width / clip.w
    scale_h = max_height / clip.h
    scale = min(scale_w, scale_h)
    return clip.resized(scale)

In [9]:
def add_margin(clip, target_width, target_height):
    top_margin = (target_height - clip.h) // 2
    bottom_margin = target_height - clip.h - top_margin
    left_margin = (target_width - clip.w) // 2
    right_margin = target_width - clip.w - left_margin
    margin = Margin(left=left_margin, right=right_margin, top=top_margin, bottom=bottom_margin, color=(0, 0, 0))
    return margin.apply(clip)

## 4. Problemas
    a) Baixa taxa de FPS (frames por segundo);
    b) Vídeos em câmera lenta.
## 4. Solução
    Acelerar o vídeo para suavizar a visualização, compensando a baixa fluidez causada pela baixa taxa de quadros.

In [10]:
def accelerate(clip, fps):
    return clip.with_fps(fps=fps, change_duration=True)

## Base de dados limpa

In [11]:
for video_params in valid_boxes:
    clip,filename = load_video(video_params, clean_labels)
    if clip:
        cropped_clip = crop(clip, video_params, filename)
        for clip,filename in cropped_clip:
            resized_clip = resize(clip, 640, 480)
            clip_with_margin = add_margin(resized_clip, 640, 480)
            # accelerated_clip = accelerate(clip_with_margin, 30)
            clip_with_margin.write_videofile(destination_directory+filename)

{'video_found': True, 'audio_found': False, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso2avc1mp41', 'encoder': 'Lavf61.1.100'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [1280, 720], 'bitrate': 1268, 'fps': 29.97002997002997, 'codec_name': 'h264', 'profile': '(High)', 'metadata': {'Metadata': '', 'handler_name': 'VideoHandler', 'vendor_id': '[0][0][0][0]', 'encoder': 'Lavc61.3.100 libx264'}}], 'input_number': 0}], 'duration': 14.05, 'bitrate': 1272, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'h264', 'video_profile': '(High)', 'video_size': [1280, 720], 'video_bitrate': 1268, 'video_fps': 29.97002997002997, 'video_duration': 14.05, 'video_n_frames': 421}
/home/pedro-antonio/Documentos/repos/projeto_mestrado_v2/myvenv/lib/python3.12/site-packages/imageio_ffmpeg/binaries/ffmpeg-linux-x86_64-v7.0.2 -i

MoviePy - Done !
MoviePy - video ready pre-processing-result/zXiwhl26DqI_0_0.mp4
{'video_found': True, 'audio_found': False, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso2avc1mp41', 'encoder': 'Lavf61.1.100'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [480, 360], 'bitrate': 224, 'fps': 20.0, 'codec_name': 'h264', 'profile': '(High)', 'metadata': {'Metadata': '', 'handler_name': 'VideoHandler', 'vendor_id': '[0][0][0][0]', 'encoder': 'Lavc61.3.100 libx264'}}], 'input_number': 0}], 'duration': 3.45, 'bitrate': 228, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'h264', 'video_profile': '(High)', 'video_size': [480, 360], 'video_bitrate': 224, 'video_fps': 20.0, 'video_duration': 3.45, 'video_n_frames': 69}
/home/pedro-antonio/Documentos/repos/projeto_mestrado_v2/myvenv/lib/python3.12/site-packages/imagei

MoviePy - Done !
MoviePy - video ready pre-processing-result/zVzXEht1aME_0.mp4
{'video_found': True, 'audio_found': False, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso2avc1mp41', 'encoder': 'Lavf61.1.100'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [480, 360], 'bitrate': 233, 'fps': 20.0, 'codec_name': 'h264', 'profile': '(High)', 'metadata': {'Metadata': '', 'handler_name': 'VideoHandler', 'vendor_id': '[0][0][0][0]', 'encoder': 'Lavc61.3.100 libx264'}}], 'input_number': 0}], 'duration': 36.2, 'bitrate': 235, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'h264', 'video_profile': '(High)', 'video_size': [480, 360], 'video_bitrate': 233, 'video_fps': 20.0, 'video_duration': 36.2, 'video_n_frames': 724}
/home/pedro-antonio/Documentos/repos/projeto_mestrado_v2/myvenv/lib/python3.12/site-packages/imageio

MoviePy - Done !
MoviePy - video ready pre-processing-result/zVzXEht1aME_1.mp4
{'video_found': True, 'audio_found': False, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso2avc1mp41', 'encoder': 'Lavf61.1.100'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [1280, 720], 'bitrate': 2799, 'fps': 29.97002997002997, 'codec_name': 'h264', 'profile': '(High)', 'metadata': {'Metadata': '', 'handler_name': 'VideoHandler', 'vendor_id': '[0][0][0][0]', 'encoder': 'Lavc61.3.100 libx264'}}], 'input_number': 0}], 'duration': 3.7, 'bitrate': 2803, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'h264', 'video_profile': '(High)', 'video_size': [1280, 720], 'video_bitrate': 2799, 'video_fps': 29.97002997002997, 'video_duration': 3.7, 'video_n_frames': 110}
/home/pedro-antonio/Documentos/repos/projeto_mestrado_v2/myvenv/lib/pyt

MoviePy - Done !
MoviePy - video ready pre-processing-result/ztPQViol2bc_1_0.mp4
{'video_found': True, 'audio_found': False, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso2avc1mp41', 'encoder': 'Lavf61.1.100'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [1280, 720], 'bitrate': 1161, 'fps': 29.97002997002997, 'codec_name': 'h264', 'profile': '(High)', 'metadata': {'Metadata': '', 'handler_name': 'VideoHandler', 'vendor_id': '[0][0][0][0]', 'encoder': 'Lavc61.3.100 libx264'}}], 'input_number': 0}], 'duration': 6.97, 'bitrate': 1164, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'h264', 'video_profile': '(High)', 'video_size': [1280, 720], 'video_bitrate': 1161, 'video_fps': 29.97002997002997, 'video_duration': 6.97, 'video_n_frames': 208}
/home/pedro-antonio/Documentos/repos/projeto_mestrado_v2/myvenv/lib

...

MoviePy - Done !
MoviePy - video ready pre-processing-result/651RQwxB3WA_63_0.mp4
{'video_found': True, 'audio_found': False, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso2avc1mp41', 'encoder': 'Lavf61.1.100'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [480, 360], 'bitrate': 214, 'fps': 25.0, 'codec_name': 'h264', 'profile': '(High)', 'metadata': {'Metadata': '', 'handler_name': 'VideoHandler', 'vendor_id': '[0][0][0][0]', 'encoder': 'Lavc61.3.100 libx264'}}], 'input_number': 0}], 'duration': 0.6, 'bitrate': 227, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'h264', 'video_profile': '(High)', 'video_size': [480, 360], 'video_bitrate': 214, 'video_fps': 25.0, 'video_duration': 0.6, 'video_n_frames': 15}
/home/pedro-antonio/Documentos/repos/projeto_mestrado_v2/myvenv/lib/python3.12/site-packages/imageio

MoviePy - Done !
MoviePy - video ready pre-processing-result/AJKLxHzIHYY_50_0.mp4
{'video_found': True, 'audio_found': False, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso2avc1mp41', 'encoder': 'Lavf61.1.100'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [1280, 720], 'bitrate': 1683, 'fps': 30.0, 'codec_name': 'h264', 'profile': '(High)', 'metadata': {'Metadata': '', 'handler_name': 'VideoHandler', 'vendor_id': '[0][0][0][0]', 'encoder': 'Lavc61.3.100 libx264'}}], 'input_number': 0}], 'duration': 3.93, 'bitrate': 1687, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'h264', 'video_profile': '(High)', 'video_size': [1280, 720], 'video_bitrate': 1683, 'video_fps': 30.0, 'video_duration': 3.93, 'video_n_frames': 117}
/home/pedro-antonio/Documentos/repos/projeto_mestrado_v2/myvenv/lib/python3.12/site-packages

MoviePy - Done !
MoviePy - video ready pre-processing-result/mH0-_I0XVdE_76_0.mp4
{'video_found': True, 'audio_found': False, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso2avc1mp41', 'encoder': 'Lavf61.1.100'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [640, 360], 'bitrate': 173, 'fps': 29.97002997002997, 'codec_name': 'h264', 'profile': '(High)', 'metadata': {'Metadata': '', 'handler_name': 'VideoHandler', 'vendor_id': '[0][0][0][0]', 'encoder': 'Lavc61.3.100 libx264'}}], 'input_number': 0}], 'duration': 5.07, 'bitrate': 177, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'h264', 'video_profile': '(High)', 'video_size': [640, 360], 'video_bitrate': 173, 'video_fps': 29.97002997002997, 'video_duration': 5.07, 'video_n_frames': 151}
/home/pedro-antonio/Documentos/repos/projeto_mestrado_v2/myvenv/lib/pyt

MoviePy - Done !
MoviePy - video ready pre-processing-result/nkFprOkNnRE_91.mp4
{'video_found': True, 'audio_found': False, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso2avc1mp41', 'encoder': 'Lavf61.1.100'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [1280, 720], 'bitrate': 1344, 'fps': 29.97002997002997, 'codec_name': 'h264', 'profile': '(High)', 'metadata': {'Metadata': '', 'handler_name': 'VideoHandler', 'vendor_id': '[0][0][0][0]', 'encoder': 'Lavc61.3.100 libx264'}}], 'input_number': 0}], 'duration': 25.96, 'bitrate': 1348, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'h264', 'video_profile': '(High)', 'video_size': [1280, 720], 'video_bitrate': 1344, 'video_fps': 29.97002997002997, 'video_duration': 25.96, 'video_n_frames': 778}
/home/pedro-antonio/Documentos/repos/projeto_mestrado_v2/myvenv/li

MoviePy - Done !
MoviePy - video ready pre-processing-result/zXiwhl26DqI_1_0.mp4
